In [61]:
import json
import pandas as pd

symbols = None
for ch in range(ord('a'), ord('z') + 1):
    letter = chr(ch)
    before = symbols.shape if symbols is not None else None
    first, last, count = 0, 5000, 50
    for start in range(first, last, count):
        path = f'../yahoo/lookup/{letter}/yahoo_symbols_{start}_{start+count}.json'
        with open(path) as f:
            js = json.load(f)
            docs = js['finance']['result'][0]['documents']
            if len(docs) != 50:
                print(f'UH OH, only got {len(docs)} records in {path}')
                break
            df = pd.DataFrame.from_records(docs)
            symbols = df if symbols is None else pd.concat([symbols, df])
            symbols = symbols.drop_duplicates(subset=['symbol'])
    after = symbols.shape
    if before is not None:
        print(f'Loaded "{letter}", {before}, deduped to {after} (diff: {after[0] - before[0]})')
symbols = symbols.set_index('symbol')
symbols.info

Loaded "b", (4263, 10), deduped to (8540, 10) (diff: 4277)
Loaded "c", (8540, 10), deduped to (12180, 10) (diff: 3640)
Loaded "d", (12180, 10), deduped to (15661, 10) (diff: 3481)
Loaded "e", (15661, 10), deduped to (18784, 10) (diff: 3123)
Loaded "f", (18784, 10), deduped to (21666, 10) (diff: 2882)
Loaded "g", (21666, 10), deduped to (24291, 10) (diff: 2625)
Loaded "h", (24291, 10), deduped to (26721, 10) (diff: 2430)
Loaded "i", (26721, 10), deduped to (28201, 10) (diff: 1480)
UH OH, only got 36 records in ../yahoo/lookup/j/yahoo_symbols_2600_2650.json
Loaded "j", (28201, 10), deduped to (29983, 10) (diff: 1782)
UH OH, only got 23 records in ../yahoo/lookup/k/yahoo_symbols_4600_4650.json
Loaded "k", (29983, 10), deduped to (33047, 10) (diff: 3064)
Loaded "l", (33047, 10), deduped to (34558, 10) (diff: 1511)
Loaded "m", (34558, 10), deduped to (36109, 10) (diff: 1551)
Loaded "n", (36109, 10), deduped to (38215, 10) (diff: 2106)
Loaded "o", (38215, 10), deduped to (39788, 10) (diff: 1

<bound method DataFrame.info of                    industryName  \
symbol                            
AAPL                 Technology   
AMZN          Consumer Cyclical   
AMD                  Technology   
GOOG     Communication Services   
F             Consumer Cyclical   
...                         ...   
IZ.V            Basic Materials   
IR75.VI                     NaN   
22Z.BE               Healthcare   
ZCT1.BE      Financial Services   
ZAL1.DU       Consumer Cyclical   

                                              industryLink  \
symbol                                                       
AAPL           https://finance.yahoo.com/sector/technology   
AMZN     https://finance.yahoo.com/sector/consumer_cycl...   
AMD            https://finance.yahoo.com/sector/technology   
GOOG     https://finance.yahoo.com/sector/communication...   
F        https://finance.yahoo.com/sector/consumer_cycl...   
...                                                    ...   
IZ.V      https:

In [57]:

symbols.index.nunique()

33047

In [62]:
symbols[symbols.index.str.contains('AAPL')]

,industryName,industryLink,regularMarketPercentChange,regularMarketChange,rank,exchange,shortName,quoteType,regularMarketPrice
symbol,,,,,,,,,
AAPL,Technology,https://finance.yahoo.com/sector/technology,"{'raw': 0.33903462, 'fmt': '0.34%'}","{'raw': 0.51000977, 'fmt': '0.51'}",172245.0,NMS,Apple Inc.,equity,"{'raw': 150.94, 'fmt': '150.94'}"
AAPL.NE,Technology,https://finance.yahoo.com/sector/technology,"{'raw': -1.3425702, 'fmt': '-1.34%'}","{'raw': -0.30999947, 'fmt': '-0.31'}",20065.0,NEO,APPLE CDR (CAD HEDGED),equity,"{'raw': 22.78, 'fmt': '22.78'}"
AAPL.MX,Technology,https://finance.yahoo.com/sector/technology,"{'raw': -1.2006527, 'fmt': '-1.20%'}","{'raw': -36.79004, 'fmt': '-36.79'}",20046.0,MEX,APPLE INC,equity,"{'raw': 3027.38, 'fmt': '3,027.38'}"
AAPL.BA,Technology,https://finance.yahoo.com/sector/technology,"{'raw': -2.5283527, 'fmt': '-2.53%'}","{'raw': -121.5, 'fmt': '-121.50'}",20026.0,BUE,APPLE INC,equity,"{'raw': 4684.0, 'fmt': '4,684.00'}"
AAPL34.SA,Technology,https://finance.yahoo.com/sector/technology,"{'raw': 0.60064054, 'fmt': '0.60%'}","{'raw': 0.47000122, 'fmt': '0.47'}",20013.0,SAO,APPLE DRN,equity,"{'raw': 78.72, 'fmt': '78.72'}"


In [63]:
stored_symbols = symbols[['industryName', 'rank', 'exchange', 'shortName', 'quoteType']]
import sqlite3 as sl
conn = sl.connect('../sqlite/lookup/symbols.db')
stored_symbols.to_sql('symbols', conn, if_exists='replace')
conn.commit()

In [13]:
all_symbols.info

<bound method DataFrame.info of               industryName                                       industryLink  \
0               Technology        https://finance.yahoo.com/sector/technology   
1        Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
2               Technology        https://finance.yahoo.com/sector/technology   
3   Communication Services  https://finance.yahoo.com/sector/communication...   
4        Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
..                     ...                                                ...   
45                     NaN                                                NaN   
46       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
47       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
48       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
49             Industrials       https://finance.yahoo.com/sector/industrials

In [14]:
all_symbols = all_symbols.drop_duplicates(subset=['symbol'])

In [15]:
all_symbols.info

<bound method DataFrame.info of               industryName                                       industryLink  \
0               Technology        https://finance.yahoo.com/sector/technology   
1        Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
2               Technology        https://finance.yahoo.com/sector/technology   
3   Communication Services  https://finance.yahoo.com/sector/communication...   
4        Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
..                     ...                                                ...   
45                     NaN                                                NaN   
46       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
47       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
48       Consumer Cyclical  https://finance.yahoo.com/sector/consumer_cycl...   
49             Industrials       https://finance.yahoo.com/sector/industrials

In [ ]:
df2.to_json('symbols_300_350.json', index=False, orient='table')

In [ ]:
import sqlite3 as sl
conn = sl.connect('../sqlite/lookup/symbols.db')
symbols.to_sql('Symbols', conn)

In [ ]:
df2.to_sql('symbols', con, index='symbol')

In [ ]:
df2

In [ ]:
df2.to_sql('symbols', con)

In [ ]:
con.execute("SELECT * FROM symbols WHERE symbol LIKE 'AEMD'")

In [ ]:
df3 = pd.read_sql('SELECT * from symbols')

In [ ]:
df3 = pd.read_sql('SELECT * from symbols', con)

In [ ]:
con.commit()

In [ ]:
con.execute('SELECT * FROM symbols where symbol LIKE ?', '%A%')

In [ ]:
con.execute('SELECT * FROM symbols where symbol LIKE "%A%"')